In [4]:
# ===============================================
# 1️⃣ IMPORT LIBRARIES
# ===============================================
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import joblib

print("✅ Libraries loaded successfully.")

# ===============================================
# 2️⃣ SET DATASET PATHS (UPDATED FOR YOUR STRUCTURE)
# ===============================================
metadata_path = "../dataset_forest/UrbanSound8K.csv"
audio_dir = "../dataset_forest/audio"

metadata = pd.read_csv(metadata_path)
print("🔹 Total audio samples:", len(metadata))

# ===============================================
# 3️⃣ RELABEL DATA (ILLEGAL = 1, NATURAL = 0)
# ===============================================
illegal_classes = ['gun_shot', 'jackhammer', 'engine_idling', 'car_horn']

metadata['label'] = metadata['class'].apply(
    lambda x: 1 if x.strip().lower() in illegal_classes else 0
)

print("🔹 Illegal samples:", metadata['label'].sum())
print("🔹 Natural samples:", (metadata['label']==0).sum())

# ===============================================
# 4️⃣ FEATURE EXTRACTION FUNCTION
# ===============================================
def extract_features(file_path):
    try:
        y, sr = librosa.load(file_path, sr=22050, mono=True)  # Load audio
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)    # Extract MFCCs
        mfcc_mean = np.mean(mfcc.T, axis=0)                   # Take mean over time
        return mfcc_mean
    except Exception as e:
        print("⚠️ Error loading:", file_path, "| Error:", e)
        return None

# ===============================================
# 5️⃣ LOOP THROUGH DATASET AND EXTRACT FEATURES
# ===============================================
features = []
labels = []

print("⏳ Extracting features from audio files... This may take several minutes.")

for idx, row in metadata.iterrows():
    file_path = os.path.join(audio_dir, 'fold'+str(row['fold']), row['slice_file_name'])
    if not os.path.exists(file_path):
        print("⚠️ Missing file:", file_path)
        continue
    mfccs = extract_features(file_path)
    if mfccs is not None:
        features.append(mfccs)
        labels.append(row['label'])

# Convert to numpy arrays
X = np.array(features)
y = np.array(labels)

print("✅ Feature extraction completed.")
print("🔹 Feature dataset shape:", X.shape)

# ===============================================
# 6️⃣ SPLIT DATASET INTO TRAIN & TEST
# ===============================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print("✅ Data split: ", X_train.shape, "train samples,", X_test.shape, "test samples.")

# ===============================================
# 7️⃣ TRAIN THE SVM CLASSIFIER
# ===============================================
print("⏳ Training SVM model...")
model = SVC(kernel='rbf', probability=True)
model.fit(X_train, y_train)
print("✅ Model training completed.")

# ===============================================
# 8️⃣ EVALUATE MODEL
# ===============================================
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("\n🎯 Model Accuracy:", accuracy)
print("\n📜 Classification Report:\n", classification_report(y_test, y_pred))

# ===============================================
# 9️⃣ SAVE THE TRAINED MODEL
# ===============================================
model_save_path = "../code/chainsaw_detection_model.pkl"
os.makedirs("../code", exist_ok=True)
joblib.dump(model, model_save_path)
print("✅ Trained model saved at:", model_save_path)


✅ Libraries loaded successfully.
🔹 Total audio samples: 8732
🔹 Illegal samples: 2803
🔹 Natural samples: 5929
⏳ Extracting features from audio files... This may take several minutes.


C:\Users\Manikrishna\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1323
  warnings.warn(
C:\Users\Manikrishna\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
C:\Users\Manikrishna\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1523
  warnings.warn(


✅ Feature extraction completed.
🔹 Feature dataset shape: (8732, 13)
✅ Data split:  (6985, 13) train samples, (1747, 13) test samples.
⏳ Training SVM model...
✅ Model training completed.

🎯 Model Accuracy: 0.7595878649112765

📜 Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.92      0.84      1186
           1       0.72      0.41      0.52       561

    accuracy                           0.76      1747
   macro avg       0.74      0.67      0.68      1747
weighted avg       0.75      0.76      0.74      1747

✅ Trained model saved at: ../code/chainsaw_detection_model.pkl
